In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os, shutil
import numpy as np


In [3]:
df = pd.read_csv("C:\\Users\\david\\OneDrive\\Inholland\\DeepLearning\\Assignment 1\\palmerpenguins_original.csv")


In [4]:
df.fillna(df.mean(), inplace=True)
filled_df = df.dropna()
encoded_df = pd.get_dummies(filled_df, columns=['sex'])
print(encoded_df)



       species     island  bill_length_mm  bill_depth_mm  flipper_length_mm  \
0       Adelie  Torgersen            39.1           18.7              181.0   
1       Adelie  Torgersen            39.5           17.4              186.0   
2       Adelie  Torgersen            40.3           18.0              195.0   
4       Adelie  Torgersen            36.7           19.3              193.0   
5       Adelie  Torgersen            39.3           20.6              190.0   
..         ...        ...             ...            ...                ...   
339  Chinstrap      Dream            55.8           19.8              207.0   
340  Chinstrap      Dream            43.5           18.1              202.0   
341  Chinstrap      Dream            49.6           18.2              193.0   
342  Chinstrap      Dream            50.8           19.0              210.0   
343  Chinstrap      Dream            50.2           18.7              198.0   

     body_mass_g  year  sex_female  sex_male  
0   

C:\Users\david\AppData\Local\Temp\ipykernel_30068\3660869709.py:1: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.fillna(df.mean(), inplace=True)


# Activation Function

In [5]:
def relu(x):
    return np.maximum(0,x)

def dydxrelu(x):
    dx = np.where(x > 0, 1, 0)
    return dx
    

# Initialize weights

In [6]:
def initialize_weights(network):
    for layer in network:
        if 'weights' in layer:
            layer['weights'] = np.random.randn(layer['inputs'], layer['nodes'])
    return network
    
network = [
    {'inputs': 2, 'nodes': 3},  # Input layer
    {'nodes': 1},                # Hidden layer
    {'nodes': 1}                 # Output layer
]

# Initialize weights for each layer
for layer in network:
    if 'inputs' in layer:
        layer['weights'] = np.random.randn(layer['inputs'], layer['nodes'])

# Print initialized weights for each layer
for i, layer in enumerate(network):
    if 'weights' in layer:
        print(f"Layer {i+1} weights:\n{layer['weights']}")

Layer 1 weights:
[[ 0.94890748  0.95285915 -0.87010843]
 [-1.11209619 -0.39128469 -0.93256338]]


# Node Class

In [7]:
class Node:
    def __init__(self, inputs, activation_function):
        self.inputs = inputs
        self.activation_function = activation_function
        self.weights = np.random.randn(inputs)
        self.bias = np.random.randn()

    def activate(self, inputs):
        weighted_sum = np.dot(inputs, self.weights) + self.bias
        return self.activation_function(weighted_sum)


# Feed Forward

In [8]:
def feed_forward(node, inputs, target):
    output = node.activate(inputs)
    loss = square_loss(output, target)
    return output, loss
 


# Backpropogation

In [9]:
def backpropagation(node, inputs, target, learning_rate):
        output = node.activate(inputs)
        error = output - target
        gradient_weights = error * inputs
        gradient_bias = error
        node.weights -= learning_rate * gradient_weights
        node.bias -= learning_rate * gradient_bias


# Loss
y= true value                                                 
y_hat = predicted value

In [10]:
def square_loss(y, y_hat):
        return np.mean((y - y_hat) ** 2)

# implementation

Partial derivative of MSE


In [11]:
# Create the neural network with one node
network = [
    {'inputs': 2, 'nodes': 1, 'activation_function': relu}
]

# Initialize weights
network = initialize_weights(network)

# Create and initialize a node
node = Node(inputs=2, activation_function=relu)

# Define inputs and target
inputs = np.array([0.5, 0.3])
target = 0.8

# Perform feed forward
output, loss = feed_forward(node, inputs, target)

# Perform backpropagation
learning_rate = 0.01
backpropagation(node, inputs, target, learning_rate)